# Logistic regression and decision tree
## Predicting the damage of an earthquake

In [ ]:
import sqlite3
import warnings

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from category_encoders import OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline, make_pipeline
import seaborn as sns
from category_encoders import OrdinalEncoder
from sklearn.tree import DecisionTreeClassifier, plot_tree

warnings.simplefilter(action="ignore", category=FutureWarning)

### Preprocessing

In [ ]:
# Instantiate SQL
%load_ext sql
%sql sqlite:////home/jovyan/nepal.sqlite

In [ ]:
# First query
%%sql
SELECT distinct(district_id)
FROM id_map
LIMIT 5

In [ ]:
# Second query
%%sql
SELECT count(*)
FROM id_map
WHERE district_id=1

In [ ]:
# Third query
%%sql
SELECT count(*)
FROM id_map
WHERE district_id=3

In [ ]:
# Final query merging all the exploratory queries to extract the data I need
%%sql
SELECT distinct(im.building_id) AS b_id, bs.*, bd.damage_grade
FROM id_map AS im
JOIN building_structure AS bs
    ON im.building_id = bs.building_id
JOIN building_damage AS bd
    ON im.building_id = bd.building_id
WHERE im.district_id = 3
LIMIT 5

In [ ]:
# To start cleaning the data, I will check the correlation between the variables, except the independent "severe_damage"
correlation = df.select_dtypes("number").drop(columns="severe_damage").corr()
sns.heatmap(correlation);

In [ ]:
# Therea are two variables highly correlated between them
print(df["severe_damage"].corr(df["height_ft_pre_eq"]))
print(df["severe_damage"].corr(df["count_floors_pre_eq"]))
# As the correlation of the second one with the independent variable is closer to 0, it has less predictive power and I'll drop it

In [ ]:
# Now I built a wrangle function that will return a dataframe cleaned from the data extracted
def wrangle(db_path):
    # Connect to database
    conn = sqlite3.connect(db_path)

    # Construct query
    query = """
            SELECT distinct(im.building_id) AS b_id, bs.*, bd.damage_grade
            FROM id_map AS im
            JOIN building_structure AS bs
                ON im.building_id = bs.building_id
            JOIN building_damage AS bd
                ON im.building_id = bd.building_id
            WHERE im.district_id = 3
            """
            
    # Read query results into DataFrame
    df = pd.read_sql(query, conn, index_col="b_id")

    # Identify leaky columns
    drop_cols = [col for col in df.columns if "post_eq" in col]

    # Add high-cardinality / redundant column
    drop_cols.append("building_id")

    # Create binary target column
    df["damage_grade"] = df["damage_grade"].str[-1].astype(int)
    df["severe_damage"] = (df["damage_grade"] > 3).astype(int)

    # Drop old target
    drop_cols.append("damage_grade")
    
    # Drop multicollinearity column
    drop_cols.append("count_floors_pre_eq")
    
    # Drop columns
    df.drop(columns=drop_cols, inplace=True)

    return df

In [ ]:
df = wrangle("/home/jovyan/nepal.sqlite")
df.head()

In [ ]:
# Let's check the class balance of the independent variable
df["severe_damage"].value_counts(normalize=True).plot(
    kind="bar", xlabel="Class", ylabel="Relative Frequency", title="Class Balance"
);
# As it can be seen, the classes are balanced

In [ ]:
# Let's check if there is any relationship between footprint size of a building and the damage it sustained in the earthquake
sns.boxplot(x="severe_damage", y="plinth_area_sq_ft", data=df)
plt.xlabel("Severe Damage")
plt.ylabel("Plinth Area [sq. ft.]")
plt.title("Karepalanchok, Plinth Area vs Building Damage");

In [ ]:
# I will check if the buildings from a certain type are more propense to have severe_damage
roof_pivot = pd.pivot_table(
    df, index="roof_type", values="severe_damage", aggfunc=np.mean
).sort_values(by="severe_damage")
roof_pivot

In [ ]:
# Once I have finished the 
target = "severe_damage"
X = df.drop(columns=target)
y = df[target]
print("X shape:", X.shape)
print("y shape:", y.shape)

### Model

In [ ]:
# Let's do the vertical split
target = "severe_damage"
X = df.drop(columns=target)
y = df[target]
print("X shape:", X.shape)
print("y shape:", y.shape)

In [ ]:
# And the vertical split
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("X_val shape:", X_val.shape)
print("y_val shape:", y_val.shape)

In [ ]:
acc_baseline = y_train.value_counts(normalize=True).max()
print("Baseline Accuracy:", round(acc_baseline, 2))
# The accuracy of the Baseline is 0.55, so let's try to beat it

In [ ]:
# I will create a model with a logistic regression for beating the baseline
model_lr = model = make_pipeline(
    OneHotEncoder(use_cat_names=True),
    LogisticRegression(max_iter=1000)
)
model.fit(X_train, y_train)

In [ ]:
# I will check the accuracy of the model with the validation set
lr_train_acc = accuracy_score(y_train, model.predict(X_train))
lr_val_acc = model.score(X_val, y_val)

print("Logistic Regression, Training Accuracy Score:", lr_train_acc)
print("Logistic Regression, Validation Accuracy Score:", lr_val_acc)
# The accuracy of both is around 65%, but it could be better

In [ ]:
# As the accuracy score could be better, I will try to change the model to a decision tree
# In addition, I will try to check which are the best hyperparameters for the model
depth_hyperparams = range(1, 16)
training_acc = []
validation_acc = []
for d in depth_hyperparams:
    model_dt = make_pipeline(
    OrdinalEncoder(), DecisionTreeClassifier(max_depth=d, random_state=42)
)
    model_dt.fit(X_train, y_train)
    training_acc.append(model_dt.score(X_train, y_train))
    # Calculate validation accuracy score and append to `training_acc`
    validation_acc.append(model_dt.score(X_val, y_val))

In [ ]:
# Let's plot the results, so that we can get the better hyperparameters
plt.plot(depth_hyperparams, training_acc, label="training")
plt.plot(depth_hyperparams, validation_acc, label="validation")
plt.xlabel("Max depth")
plt.ylabel("Accuracy score")
plt.legend();

In [ ]:
# The final model will have a max_depth of 10
final_model_dt = make_pipeline(
    OrdinalEncoder(), DecisionTreeClassifier(max_depth=10, random_state=42)
)
final_model_dt.fit(X_train, y_train)

In [ ]:
# Now let's read the test to see if the model performs well
X_test = pd.read_csv("data/kavrepalanchok-test-features.csv", index_col="b_id")
y_test_pred = final_model_dt.predict(X_test)
y_test_pred[:5]

In [ ]:
# Let's get the importances of the features 
features = X_train.columns
importances = final_model_dt.named_steps["decisiontreeclassifier"].feature_importances_
feat_imp = pd.Series(importances, index=features).sort_values()
feat_imp.head()

In [ ]:
# I represent them in an horizontal bar chart
# Create horizontal bar chart of feature importances
feat_imp.plot(kind="barh")
plt.xlabel("Gini Importance")
plt.ylabel("Feature");

In [ ]:
# As it can be seen, the feature roof_type is the most important feature (by far) affecting severe damage after an earthquake